In [ ]:
using DifferentialEquations
using ODEInterfaceDiffEq
using Plots
using Plots.PlotMeasures
##plotlyjs()

# Complete model: fuel consumption and no FK-linearization

$$
\left\{\begin{aligned}
\partial_\tau \theta - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} \theta & = 
\exp\Big(\frac{\theta}{1+\theta/\beta}\Big)\, Y\\
\partial_\tau Y - {\displaystyle 1 \over \displaystyle \lambda} \partial_{zz} Y & = - \varepsilon \exp\Big(\frac{\theta}{1+\theta/\beta}\Big)\, Y
\end{aligned}\right.
$$

In [ ]:
function thermal_explosion!(du, u, p, t)
  nx, dx, lambda, eps, beta = p   
            
  theta = view(u,1:nx)
  Y     = view(u,nx+1:2*nx+2)
    
  dtheta = view(du,1:nx)
  dY     = view(du,nx+1:2*nx+2)
    
  cst = 1.0/(lambda*dx*dx)
        
  dtheta[1]      = exp(theta[1]/(1 + theta[1]/beta))*Y[2] + cst * (-2*theta[1] + theta[2])
  dtheta[2:nx-1] = exp.(theta[2:nx-1]./(1 .+ theta[2:nx-1]./beta)).*Y[3:nx] + cst * (theta[1:nx-2] - 2*theta[2:nx-1] + theta[3:nx])
  dtheta[nx]     = exp(theta[nx]/(1 + theta[nx]/beta))*Y[nx+1] + cst * (theta[nx-1] - 2*theta[nx])

  dY[1]      = -eps*exp(0/(1 + 0/beta))*Y[1] + cst * (-2*Y[1] + 2*Y[2])
  dY[2:nx+1] = -eps*exp.(theta[1:nx]./(1 .+ theta[1:nx]./beta)).*Y[2:nx+1] + cst * (Y[1:nx] - 2*Y[2:nx+1] + Y[3:nx+2])
  dY[nx+2]   = -eps*exp(0/(1 + 0/beta))*Y[nx+2] + cst * (2*Y[nx+1] - 2*Y[nx+2])
    
end;

In [ ]:
xmin = 0.
xmax = 2.0
eps = 0.002
lambda = 0.95
beta = 30

nx = 99
dx = (xmax-xmin) / (nx+1)

tini = 0.
tend = 10.7928

tspan = (tini, tend)

x = collect(range(xmin, xmax, length=nx+2))

uini = zeros(2*nx+2)
uini[nx+1:2*nx+2] = ones(nx+2)

p = (nx, dx, lambda, eps, beta)

tol = 1e-6
prob = ODEProblem(thermal_explosion!, uini, tspan, p)
sol = solve(prob, RadauIIA5(), abstol=tol, reltol=tol)

println("Number of function evaluations : ", sol.destats.nf)
println("Number of linear solves :        ", sol.destats.nsolve)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

theta = [0;view(sol[end],1:nx);0]
Y     = view(sol[end],nx+1:2*nx+2)

dt = sol.t[2:end]-sol.t[1:end-1]

ptheta = plot(x, theta, seriestype=:scatter, size=(900,900), leg=false, xaxis = ("x"), bottom_margin=10mm, 
          title="Approximate solution theta at time $tend", titlefontsize=12, markershape=:xcross, markersize=2)
pY = plot(x, Y, seriestype=:scatter, size=(900,900), leg=false, xaxis = ("x"), bottom_margin=10mm,
          title="Approximate solution Y at time $tend", titlefontsize=12, markershape=:xcross, markersize=2)
pdt = bar(sol.t[1:end-1]+dt/2, dt, bar_width=dt, size=(900,900), leg=false, yaxis=:log,
          title="Time steps")
plot(ptheta, pY, pdt, layout=(3,1))